Exploração dos dados

In [80]:
import pandas as pd
import numpy as np

# Carregando o conjunto de dados
data = pd.read_csv('./matches_brasileirao_serie_a_2022.csv')

# verficando a quantidade de instâncias
instances = data.shape[0]
print('Número de instâncias: ' + str(instances))

# verificando a quantidade de atributos
attributes = data.shape[1]
print('Número de atributos: ' + str(attributes))

# Verificando a presença de dados ausentes
print(data.isna().sum())


Número de instâncias: 380
Número de atributos: 26
match_id                    0
stage                       0
date                        0
team_name_home              0
team_name_away              0
team_home_score             0
team_away_score             0
possession_home             0
possession_away             0
total_shots_home            0
total_shots_away            0
shots_on_target_home        0
shots_on_target_away        0
duels_won_home              0
duels_won_away              0
prediction_team_home_win    0
prediction_draw             0
prediction_team_away_win    0
prediction_quantity         0
location                    0
lineup_home                 0
lineup_away                 0
player_names_home           0
player_numbers_home         0
player_names_away           0
player_numbers_away         0
dtype: int64


Criando variáveis

In [81]:
# Separando atributos desejáveis
parsed_data = data.drop(
  labels=['stage', 'date', 'location', 'lineup_home', 'lineup_away', 'player_names_home', 'player_numbers_home', 'player_names_away', 'player_numbers_away', 'prediction_team_home_win', 'prediction_draw', 'prediction_team_away_win', 'prediction_quantity'],
  axis=1)

parsed_data['both_scores_nonzero'] = np.where((data['team_home_score'] != 0) & (data['team_away_score']!=0), 1, 0)

# Criando a variável 'gols_marcados_temporada' para contabilziar a quantidade de gols marcados na tmeporada
parsed_data['gols_marcados_temporada'] = parsed_data.groupby('team_name_home')['team_home_score'].cumsum()

# Criando a variável 'gols_sofridos_temporada' para contabilziar a quantidade de gols sofridos na tmeporada
parsed_data['gols_sofridos_temporada'] = parsed_data.groupby('team_name_home')['team_away_score'].cumsum()

print(parsed_data[['team_name_home', 'team_home_score', 'gols_marcados_temporada', 'gols_sofridos_temporada']].head())

  team_name_home  team_home_score  gols_marcados_temporada   
0     América MG                1                        1  \
1  RB Bragantino                0                        0   
2  Internacional                3                        3   
3          Goiás                0                        0   
4         Cuiabá                2                        2   

   gols_sofridos_temporada  
0                        1  
1                        1  
2                        0  
3                        4  
4                        1  


Dividindo os dados em um conjunto de treinamento e testes (80% por 20%)

In [82]:
from sklearn.model_selection import train_test_split

# Divisão dos dados em conjunto de treinamento e teste
train_data, test_data = train_test_split(parsed_data, test_size=0.2, random_state=42)

# Verificando o tamanho dos conjuntos de treinamento e teste
print("Tamanho do conjunto de treinamento:", len(train_data))
print("Tamanho do conjunto de teste:", len(test_data))

Tamanho do conjunto de treinamento: 304
Tamanho do conjunto de teste: 76


Definindo o algotimo de treinamento

In [83]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Definindo as variáveis de entrada (features) e o alvo (target)
features = ['gols_marcados_temporada', 'gols_sofridos_temporada']
target = 'both_scores_nonzero'

# Separando as features e o alvo do conjunto de treinamento e teste
X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]

# Criando e treinando o modelo Decision Tree
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = model.predict(X_test)

# Calculando a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia do modelo:", accuracy)

Acurácia do modelo: 0.5657894736842105


Validando o modelo

In [84]:
from sklearn.model_selection import cross_val_score

# Aplicando validação cruzada
scores = cross_val_score(model, X_train, y_train, cv=5)

# Exibindo a acurácia média e desvio padrão
print("Acurácia média da validação cruzada:", scores.mean())
print("Desvio padrão da validação cruzada:", scores.std())

Acurácia média da validação cruzada: 0.47693989071038245
Desvio padrão da validação cruzada: 0.05448983489021858


In [85]:
# Dados de teste ou novos dados para prever
new_data = test_data 

# Aplicando as mesmas etapas de pré-processamento nas novas entradas
# ...
# Pré-processamento dos novos dados

# Realizando as previsões
new_features = new_data[features]
predictions = model.predict(new_features)

# Exibindo as previsões
print("Previsões:")
for i, pred in enumerate(predictions):
    print("Amostra", i+1, ": Ambos marcam" if pred == 1 else "Ambos não marcam")

predictions

Previsões:
Amostra 1 : Ambos marcam
Amostra 2 Ambos não marcam
Amostra 3 : Ambos marcam
Amostra 4 Ambos não marcam
Amostra 5 : Ambos marcam
Amostra 6 Ambos não marcam
Amostra 7 : Ambos marcam
Amostra 8 : Ambos marcam
Amostra 9 Ambos não marcam
Amostra 10 Ambos não marcam
Amostra 11 : Ambos marcam
Amostra 12 : Ambos marcam
Amostra 13 : Ambos marcam
Amostra 14 Ambos não marcam
Amostra 15 : Ambos marcam
Amostra 16 : Ambos marcam
Amostra 17 : Ambos marcam
Amostra 18 Ambos não marcam
Amostra 19 Ambos não marcam
Amostra 20 : Ambos marcam
Amostra 21 Ambos não marcam
Amostra 22 Ambos não marcam
Amostra 23 : Ambos marcam
Amostra 24 : Ambos marcam
Amostra 25 : Ambos marcam
Amostra 26 : Ambos marcam
Amostra 27 : Ambos marcam
Amostra 28 : Ambos marcam
Amostra 29 : Ambos marcam
Amostra 30 Ambos não marcam
Amostra 31 Ambos não marcam
Amostra 32 : Ambos marcam
Amostra 33 Ambos não marcam
Amostra 34 Ambos não marcam
Amostra 35 Ambos não marcam
Amostra 36 Ambos não marcam
Amostra 37 Ambos não marcam
Am

array([1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0])